In [ ]:
# fMRI Demo

## Introduction

Here we show how to load your BIDS-structured fMRI dataset into rsatoolbox. n14 2ses

![stimuli](demo_fmri_files/mur32_stims.png)

This library depends on *nibabel* for convenience functions to access MRI data, and is used separately in the demo as well, so start by installing nibabel (`pip install nibabel`).

Then run the following imports:

In [ ]:
%matplotlib inline
from os.path import expanduser
from rsatoolbox.io.fmriprep import find_fmriprep_runs
from rsatoolbox.data.dataset import Dataset

In [ ]:
data_dir = expanduser('~/data/rsatoolbox/mur32')

![task design](demo_fmri_files/mur32_tasks.png)